In [1]:
import pandas as pd
import numpy
import re
!pip install biopython
import Bio
from Bio import Entrez
Entrez.email = "yapyuyan@gmail.com"     # Always tell NCBI who you are


from itertools import product

In [2]:
Find =['MTRNR2L8','PPBP','ACTB','S100A9','TMSB4X','S100A11','FTL','GSE1', 'TXLNGY','exo_circ_22106','exo_circ_79050','hsa_circ_0000722','hsa_circ_0009024','hsa_circRNA_009024','circGSE1','circTXLNGY']


find1=[term + ' AND HCC' for term in Find ]

find2=[term + ' AND hepatocellular carcinoma' for term in Find ]

find3=[term + ' AND LIHC' for term in Find ]

Find1=find1+find2+find3





In [3]:
Combination=pd.DataFrame(list(product(Find,Find2)))
Find3=Combination[0]+" AND " +Combination[1]
Find3

NameError: name 'Find2' is not defined

In [4]:
pubmedsearch = []

for i in Find1:
    handle = Entrez.esearch(db="pubmed", term= i, retmax=5)
    record = Entrez.read(handle)
    if len(record["IdList"])>0:
        pubmedsearch.append (
            {'feature': i,
             'pubmedID':record["IdList"],})
   
first_search=pd.DataFrame(pubmedsearch)


df_explode = first_search.explode('pubmedID')
df_explode= df_explode.sort_values(by='pubmedID').reset_index(drop=True)
print(df_explode)
 
Abstracts=[]

def get_abstract(pmid):
   
    handle = Entrez.efetch(db='pubmed', id=pmid, retmode='text', rettype='abstract')
    return handle.read()

for n in df_explode['pubmedID']:
   
    Abstracts.append (
           {
            'pubmedID': n,
            'Abstract':get_abstract(n),
            }
           )
   
Abstracts=pd.DataFrame(Abstracts)
Abstracts= Abstracts.sort_values(by='pubmedID').reset_index()



                                  feature  pubmedID
0     TMSB4X AND hepatocellular carcinoma  24299315
1                          TMSB4X AND HCC  24299315
2     TMSB4X AND hepatocellular carcinoma  29221787
3                          TMSB4X AND HCC  29221787
4                         S100A11 AND HCC  29568350
5                            PPBP AND HCC  31346321
6                             FTL AND HCC  31432149
7                         S100A11 AND HCC  31703598
8    S100A11 AND hepatocellular carcinoma  31703598
9                            ACTB AND HCC  31846694
10   S100A11 AND hepatocellular carcinoma  31919231
11                        S100A11 AND HCC  31919231
12                            FTL AND HCC  32577166
13       FTL AND hepatocellular carcinoma  32577166
14                           ACTB AND HCC  33465003
15      ACTB AND hepatocellular carcinoma  33465003
16                        S100A11 AND HCC  33546025
17   S100A11 AND hepatocellular carcinoma  33546025
18   S100A11

In [5]:
Compiled= pd.DataFrame(df_explode.join(Abstracts['Abstract']))

In [11]:
Compiled

,feature,Abstract
pubmedID,,
24299315,"TMSB4X AND hepatocellular carcinoma,TMSB4X AND...",\n1. Neoplasma. 2014;61(2):193-204. doi: 10.41...
29221787,"TMSB4X AND hepatocellular carcinoma,TMSB4X AND...",\n1. Infect Genet Evol. 2018 Mar;58:17-22. doi...
29568350,S100A11 AND HCC,\n1. Oncotarget. 2018 Jan 11;9(17):13193-13205...
31346321,PPBP AND HCC,\n1. Cancer Cell Int. 2019 Jul 18;19:186. doi:...
31432149,FTL AND HCC,\n1. Int J Oncol. 2019 Oct;55(4):860-878. doi:...
31703598,"S100A11 AND HCC,S100A11 AND hepatocellular car...",\n1. J Hematol Oncol. 2019 Nov 8;12(1):112. do...
31846694,ACTB AND HCC,\n1. Cell Signal. 2020 Mar;67:109479. doi: 10....
31919231,"S100A11 AND hepatocellular carcinoma,S100A11 A...",\n1. Gut. 2020 Oct;69(10):1841-1854. doi: 10.1...
32577166,"FTL AND HCC,FTL AND hepatocellular carcinoma",\n1. Oncotarget. 2020 Jun 9;11(23):2216-2232. ...


In [14]:
set(Compiled['feature'])
Compiled.index
#list(set(Find1)-set(Compiled['feature']))

Index(['24299315', '29221787', '29568350', '31346321', '31432149', '31703598',
       '31846694', '31919231', '32577166', '33465003', '33546025', '33571540',
       '33815482', '33864445', '33866937', '34584568', '34665400', '34790741',
       '35378780', '35396771', '35397536', '35435804', '35560794', '35651950',
       '35693827', '35992798', '36032071', '36041055'],
      dtype='object', name='pubmedID')

In [15]:

Final=Compiled.groupby(Compiled.index).aggregate({'feature':','.join,'Abstract':'first'})
Final.to_excel('FeatureSearch.xlsx')


In [4]:
pubmedsearch = []

for i in Find2:
    handle = Entrez.esearch(db="pubmed", term= i, retmax=100)
    record = Entrez.read(handle)
    if len(record["IdList"])>0:
        pubmedsearch.append (
            {'feature': i,
             'pubmedID':record["IdList"],})
   
first_search=pd.DataFrame(pubmedsearch)


df_explode = first_search.explode('pubmedID')
df_explode= df_explode.sort_values(by='pubmedID').reset_index(drop=True)
print(df_explode)
 
Abstracts=[]

def get_abstract(pmid):
   
    handle = Entrez.efetch(db='pubmed', id=pmid, retmode='text', rettype='abstract')
    return handle.read()

for n in df_explode['pubmedID']:
   
    Abstracts.append (
           {
            'pubmedID': n,
            'Abstract':get_abstract(n),
            }
           )
   
Abstracts=pd.DataFrame(Abstracts)
Abstracts= Abstracts.sort_values(by='pubmedID').reset_index()




              feature  pubmedID
0  hsa_circRNA_009024  29448254
1    hsa_circ_0009024  30214434
2            circGSE1  33408484
3    hsa_circ_0009024  33955043
4            circGSE1  35366240
5            circGSE1  35396771
6            circGSE1  35855756


In [5]:
Compiled= pd.DataFrame(df_explode.join(Abstracts['Abstract']))
Compiled=Compiled.groupby('pubmedID').agg({'feature':','.join,'Abstract':'first'})
Compiled.to_excel('HCC_circRNAsearch.xlsx')


In [7]:
pubmedsearch = []

for i in Find3:
    handle = Entrez.esearch(db="pubmed", term= i, retmax=100)
    record = Entrez.read(handle)
    if len(record["IdList"])>0:
        pubmedsearch.append (
            {'feature': i,
             'pubmedID':record["IdList"],})
   
first_search=pd.DataFrame(pubmedsearch)


df_explode = first_search.explode('pubmedID')
df_explode= df_explode.sort_values(by='pubmedID').reset_index(drop=True)
print(df_explode)
 
Abstracts=[]

def get_abstract(pmid):
   
    handle = Entrez.efetch(db='pubmed', id=pmid, retmode='text', rettype='abstract')
    return handle.read()

for n in df_explode['pubmedID']:
   
    Abstracts.append (
           {
            'pubmedID': n,
            'Abstract':get_abstract(n),
            }
           )
   
Abstracts=pd.DataFrame(Abstracts)
Abstracts= Abstracts.sort_values(by='pubmedID').reset_index()
Compiled= pd.DataFrame(df_explode.join(Abstracts['Abstract']))
Compiled=Compiled.groupby('pubmedID').agg({'feature':','.join,'Abstract':'first'})



KeyError: 'pubmedID'

In [ ]:
first_search